In [ ]:
!pip install qiskit qiskit-machine-learning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.6/231.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 8.5 MB/s eta 0:00:00


In [ ]:
from qiskit import QuantumCircuit
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit.primitives import Estimator

In [ ]:
# Inisialisasi estimator
estimator = Estimator()

# Fungsi untuk membuat Quantum Neural Network (QNN)
def qnn1():
    feature_map = ZZFeatureMap(2)
    ansatz = RealAmplitudes(2, reps=1)
    qc = QuantumCircuit(2)
    qc.compose(feature_map, inplace=True)
    qc.compose(ansatz, inplace=True)

    # Mengaktifkan input_gradients untuk backprop hybrid gradient
    qnn = EstimatorQNN(
        circuit=qc,
        input_params=feature_map.parameters,
        weight_params=ansatz.parameters,
        input_gradients=True,
        estimator=estimator,
    )
    return qnn

# Membuat QNN
qnn4 = qnn1()

<ipython-input-11-fe9bf4fc5da2>:2: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()
<ipython-input-11-fe9bf4fc5da2>:13: DeprecationWarning: V1 Primitives are deprecated as of qiskit-machine-learning 0.8.0 and will be removed no sooner than 4 months after the release date. Use V2 primitives for continued compatibility and support.
  qnn = EstimatorQNN(


In [ ]:
qnn4

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from qiskit import QuantumCircuit
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit.primitives import Estimator

class GPTBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion):
        super(GPTBlock, self).__init__()
        self.attention = nn.MultiheadAttention(embed_dim=embed_size, num_heads=heads, batch_first=True)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size)
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        print("\n===== FORWARD PASS GPTBlock =====")
        print(f"Input tensor shape: {x.shape}")

        attn_output, _ = self.attention(x, x, x, attn_mask=mask)
        print(f"Attention output shape: {attn_output.shape}")
        print(f"Attention output values:\n {attn_output}")

        x = self.norm1(x + self.dropout(attn_output))
        print(f"After LayerNorm1 shape: {x.shape}")
        print(f"After LayerNorm1 values:\n {x}")

        forward_out = self.feed_forward(x)
        print(f"Feed Forward output shape: {forward_out.shape}")
        print(f"Feed Forward output values:\n {forward_out}")

        out = self.norm2(x + self.dropout(forward_out))
        print(f"After LayerNorm2 shape: {out.shape}")
        print(f"After LayerNorm2 values:\n {out}")

        print("=================================\n")
        return out

    def qnn1():
      feature_map = ZZFeatureMap(2)
      ansatz = RealAmplitudes(2, reps=1)
      qc = QuantumCircuit(2)
      qc.compose(feature_map, inplace=True)
      qc.compose(ansatz, inplace=True)

      # Mengaktifkan input_gradients untuk backprop hybrid gradient
      qnn = EstimatorQNN(
          circuit=qc,
          input_params=feature_map.parameters,
          weight_params=ansatz.parameters,
          input_gradients=True,
          estimator=estimator,
      )
      return qnn

class CustomGPT(nn.Module):
    def __init__(self, vocab_size, embed_size, num_layers, heads, device, forward_expansion, dropout, max_length):
        super(CustomGPT, self).__init__()
        self.device = device
        self.embed_size = embed_size
        self.word_embedding = nn.Embedding(vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)
        self.layers = nn.ModuleList([
            GPTBlock(embed_size, heads, dropout, forward_expansion) for _ in range(num_layers)
        ])
        self.fc_out = nn.Linear(embed_size, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length, device=self.device).unsqueeze(0)
        out = self.dropout(self.word_embedding(x) + self.position_embedding(positions))
        for layer in self.layers:
            out = layer(out, mask)
        out = self.fc_out(out)
        return out

    def generate_text(self, input_ids, max_length):
        self.eval()
        for _ in range(max_length):
            seq_length = input_ids.shape[1]
            mask = causal_mask(seq_length, self.device)
            logits = self.forward(input_ids, mask)
            next_token = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
            input_ids = torch.cat([input_ids, next_token], dim=1)
        return input_ids

def causal_mask(seq_length, device):
    """Membuat causal mask dengan ukuran yang sesuai"""
    mask = torch.tril(torch.ones((seq_length, seq_length), device=device))
    return mask.masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))


# Fixed QPT Model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from qiskit import QuantumCircuit
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit.primitives import Estimator

class QPTBlock(nn.Module):
    def __init__(self, embed_size, heads, dropout, forward_expansion, reps=1):
        super(QPTBlock, self).__init__()
        self.attention = nn.MultiheadAttention(embed_dim=embed_size, num_heads=heads, batch_first=True)
        self.norm1 = nn.LayerNorm(embed_size)
        self.norm2 = nn.LayerNorm(embed_size)
        self.norm3 = nn.LayerNorm(embed_size)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_size, forward_expansion * embed_size),
            nn.ReLU(),
            nn.Linear(forward_expansion * embed_size, embed_size)
        )
        self.reps = reps
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        batch_size, seq_length, feature_dim = x.shape
        attn_output, _ = self.attention(x, x, x, attn_mask=mask)
        x = self.norm1(x + self.dropout(attn_output))
        forward_out = self.feed_forward(x)
        out = self.norm2(x + self.dropout(forward_out))
        q_out = self.qnn1(x)
        q_norm = self.norm2(x + self.dropout(q_out))
        return q_norm

    def qnn1(self, x):
        batch_size, seq_length, feature_dim = x.shape
        num_features = feature_dim
        num_qubits = feature_dim
        estimator = StatevectorEstimator()

        feature_map = ZZFeatureMap(num_qubits)
        ansatz = RealAmplitudes(num_qubits, reps=self.reps)

        # Quantum Circuit
        qc = QuantumCircuit(num_qubits)
        qc.compose(feature_map, inplace=True)
        qc.compose(ansatz, inplace=True)

        # Definisi QNN
        qnn = EstimatorQNN(
            circuit=qc,
            input_params=feature_map.parameters,
            weight_params=ansatz.parameters,
            input_gradients=True,
            estimator=estimator,
        )

        # Menghubungkan dengan PyTorch menggunakan TorchConnector
        qnn_layer = TorchConnector(qnn)

        x = qnn_layer(x)

        return x

class QPT(nn.Module):
    def __init__(self, vocab_size, embed_size, num_layers, heads, device, forward_expansion, dropout, max_length):
        super(QPT, self).__init__()
        self.device = device
        self.embed_size = embed_size
        self.word_embedding = nn.Embedding(vocab_size, embed_size)
        self.position_embedding = nn.Embedding(max_length, embed_size)
        self.layers = nn.ModuleList([
            QPTBlock(embed_size, heads, dropout, forward_expansion) for _ in range(num_layers)
        ])
        self.fc_out = nn.Linear(embed_size, vocab_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        N, seq_length = x.shape
        positions = torch.arange(0, seq_length, device=self.device).unsqueeze(0)
        out = self.dropout(self.word_embedding(x) + self.position_embedding(positions))
        for layer in self.layers:
            out = layer(out, mask)
        out = self.fc_out(out)
        return out

    def generate_text(self, input_ids, max_length):
        self.eval()
        for _ in range(max_length):
            seq_length = input_ids.shape[1]
            mask = causal_mask(seq_length, self.device)
            logits = self.forward(input_ids, mask)
            next_token = torch.argmax(logits[:, -1, :], dim=-1, keepdim=True)
            input_ids = torch.cat([input_ids, next_token], dim=1)
        return input_ids

def causal_mask(seq_length, device):
    """Membuat causal mask dengan ukuran yang sesuai"""
    mask = torch.tril(torch.ones((seq_length, seq_length), device=device))
    return mask.masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))

In [ ]:
class SimpleTokenizer:
    def __init__(self, vocab):
        self.vocab = {word: idx for idx, word in enumerate(vocab)}
        self.inv_vocab = {idx: word for word, idx in self.vocab.items()}

    def encode(self, text):
        return torch.tensor([[self.vocab.get(word, self.vocab["<UNK>"])] for word in text.split()], dtype=torch.long)

    def decode(self, tokens):
        return " ".join([self.inv_vocab.get(int(token), "<UNK>") for token in tokens])

In [ ]:
from transformers import Pipeline
import torch

class QPTPipeline(Pipeline):
    def __init__(self, model, tokenizer, device="cpu"):
        self.model = model.to(device)
        self.tokenizer = tokenizer
        self.device = device

    def _forward(self, inputs):
        """Proses forward dari pipeline."""
        tokens = self.tokenizer.encode(inputs)
        tokens = tokens.to(self.device)
        mask = causal_mask(tokens.shape[1], self.device)

        with torch.no_grad():
            output = self.model(tokens, mask)

        output_tokens = torch.argmax(output, dim=-1).squeeze().tolist()
        generated_text = self.tokenizer.decode(output_tokens)

        return {"generated_text": generated_text}


In [ ]:
# Buat vocab sederhana
vocab = ["hello", "world", "this", "is", "a", "test", "<PAD>", "<UNK>"]
tokenizer = SimpleTokenizer(vocab)

# QPT
device = "cuda" if torch.cuda.is_available() else "cpu"
model = QPT(
    vocab_size=len(vocab),
    embed_size=16,
    num_layers=2,
    heads=2,
    device=device,
    forward_expansion=4,
    dropout=0.1,
    max_length=10
).to(device)

# Contoh input
text = "hello world"
input_ids = tokenizer.encode(text).T.to(device)

# Generate teks baru
generated_ids = model.generate_text(input_ids, max_length=5)
generated_text = tokenizer.decode(generated_ids.squeeze(0).tolist())

print("Generated Text:", generated_text)



===== FORWARD PASS GPTBlock =====
Input tensor shape: torch.Size([1, 2, 16])
Attention output shape: torch.Size([1, 2, 16])
Attention output values:
 tensor([[[-0.0173, -0.0520,  0.0578,  0.1463, -0.3406, -0.0839,  0.0198,
           0.4546,  0.0734, -0.5396, -0.3322,  0.3096,  0.6608, -0.3068,
           0.8802,  0.2424],
         [-0.4089, -0.2677, -0.0553, -0.0294, -0.0931,  0.1733,  0.0309,
           0.1977,  0.3444, -0.0926, -0.2511,  0.1161,  0.0842, -0.0959,
           0.0996, -0.0221]]], grad_fn=<TransposeBackward0>)
After LayerNorm1 shape: torch.Size([1, 2, 16])
After LayerNorm1 values:
 tensor([[[-0.4416,  0.4411, -0.6206,  1.6393, -0.0280, -0.8981, -1.9141,
           0.8860,  0.3474, -0.4151,  1.3381, -0.3884, -1.0729, -0.9800,
           0.4546,  1.6520],
         [-0.4905,  1.3870,  0.2051, -0.1102, -0.8697,  1.2020,  0.9405,
          -0.3574, -1.0092,  0.0109, -0.5256, -1.8664,  2.0913,  0.7141,
          -0.5338, -0.7881]]], grad_fn=<NativeLayerNormBackward0>)
Feed F



===== FORWARD PASS GPTBlock =====
Input tensor shape: torch.Size([1, 3, 16])
Attention output shape: torch.Size([1, 3, 16])
Attention output values:
 tensor([[[-0.0173, -0.0520,  0.0578,  0.1463, -0.3406, -0.0839,  0.0198,
           0.4546,  0.0734, -0.5396, -0.3322,  0.3096,  0.6608, -0.3068,
           0.8802,  0.2424],
         [-0.4089, -0.2677, -0.0553, -0.0294, -0.0931,  0.1733,  0.0309,
           0.1977,  0.3444, -0.0926, -0.2511,  0.1161,  0.0842, -0.0959,
           0.0996, -0.0221],
         [-0.6048,  0.3803,  0.5669,  0.3687,  0.1840,  0.4337,  0.3551,
           0.2703, -0.2131,  0.0049, -0.0772, -0.1377,  0.0443, -0.2824,
          -0.2769,  0.5257]]], grad_fn=<TransposeBackward0>)
After LayerNorm1 shape: torch.Size([1, 3, 16])
After LayerNorm1 values:
 tensor([[[-0.4416,  0.4411, -0.6206,  1.6393, -0.0280, -0.8981, -1.9141,
           0.8860,  0.3474, -0.4151,  1.3381, -0.3884, -1.0729, -0.9800,
           0.4546,  1.6520],
         [-0.4905,  1.3870,  0.2051, -0.110



===== FORWARD PASS GPTBlock =====
Input tensor shape: torch.Size([1, 3, 16])
Attention output shape: torch.Size([1, 3, 16])
Attention output values:
 tensor([[[-0.9281, -0.6185, -0.6200,  0.0637,  0.2042,  0.7953,  0.4934,
          -0.8538, -0.2729,  0.3200,  0.3530, -0.0619,  0.5641, -0.3844,
           0.4562,  0.0455],
         [ 0.0887, -0.4910, -0.3405,  0.2276, -0.0073,  0.4907,  0.1419,
          -0.2056, -0.2369,  0.2991,  0.0270,  0.2408,  0.3458, -0.2494,
          -0.1236,  0.4249],
         [ 0.2936, -0.1846, -0.1150,  0.0176,  0.2602, -0.0943, -0.0831,
          -0.1513, -0.2719,  0.2654, -0.0218,  0.2148, -0.1572,  0.0498,
          -0.2274,  0.2091]]], grad_fn=<TransposeBackward0>)
After LayerNorm1 shape: torch.Size([1, 3, 16])
After LayerNorm1 values:
 tensor([[[-1.2903, -0.1438, -1.1662,  1.6642,  0.1961, -0.0722, -1.3394,
           0.0577,  0.0984, -0.0648,  1.6528, -0.4063, -0.4625, -1.2853,
           0.9026,  1.6590],
         [-0.4238,  0.8224, -0.1679,  0.074



===== FORWARD PASS GPTBlock =====
Input tensor shape: torch.Size([1, 4, 16])
Attention output shape: torch.Size([1, 4, 16])
Attention output values:
 tensor([[[-0.0173, -0.0520,  0.0578,  0.1463, -0.3406, -0.0839,  0.0198,
           0.4546,  0.0734, -0.5396, -0.3322,  0.3096,  0.6608, -0.3068,
           0.8802,  0.2424],
         [-0.4089, -0.2677, -0.0553, -0.0294, -0.0931,  0.1733,  0.0309,
           0.1977,  0.3444, -0.0926, -0.2511,  0.1161,  0.0842, -0.0959,
           0.0996, -0.0221],
         [-0.6048,  0.3803,  0.5669,  0.3687,  0.1840,  0.4337,  0.3551,
           0.2703, -0.2130,  0.0049, -0.0772, -0.1377,  0.0443, -0.2824,
          -0.2769,  0.5257],
         [-0.0878,  0.4524,  0.3639,  0.3821,  0.2771, -0.1935,  0.1082,
          -0.0107, -0.4040, -0.0544, -0.0787, -0.1890,  0.3107, -0.3292,
           0.2937,  0.6249]]], grad_fn=<TransposeBackward0>)
After LayerNorm1 shape: torch.Size([1, 4, 16])
After LayerNorm1 values:
 tensor([[[-0.4416,  0.4411, -0.6206,  1.639



===== FORWARD PASS GPTBlock =====
Input tensor shape: torch.Size([1, 4, 16])
Attention output shape: torch.Size([1, 4, 16])
Attention output values:
 tensor([[[-0.9281, -0.6185, -0.6200,  0.0637,  0.2042,  0.7953,  0.4934,
          -0.8538, -0.2729,  0.3200,  0.3530, -0.0619,  0.5641, -0.3844,
           0.4562,  0.0455],
         [ 0.0887, -0.4910, -0.3405,  0.2276, -0.0073,  0.4907,  0.1419,
          -0.2056, -0.2369,  0.2991,  0.0270,  0.2408,  0.3458, -0.2494,
          -0.1236,  0.4249],
         [ 0.2936, -0.1846, -0.1150,  0.0176,  0.2602, -0.0943, -0.0831,
          -0.1513, -0.2719,  0.2654, -0.0218,  0.2148, -0.1572,  0.0498,
          -0.2274,  0.2091],
         [-0.2837, -0.0567, -0.2609, -0.1931,  0.3167, -0.1006, -0.0679,
          -0.3130, -0.2458,  0.1872,  0.2087,  0.0036, -0.3275,  0.2212,
           0.0785, -0.0915]]], grad_fn=<TransposeBackward0>)
After LayerNorm1 shape: torch.Size([1, 4, 16])
After LayerNorm1 values:
 tensor([[[-1.2903, -0.1438, -1.1662,  1.664



===== FORWARD PASS GPTBlock =====
Input tensor shape: torch.Size([1, 5, 16])
Attention output shape: torch.Size([1, 5, 16])
Attention output values:
 tensor([[[-0.0173, -0.0520,  0.0578,  0.1463, -0.3406, -0.0839,  0.0198,
           0.4546,  0.0734, -0.5396, -0.3322,  0.3096,  0.6608, -0.3068,
           0.8802,  0.2424],
         [-0.4089, -0.2677, -0.0553, -0.0294, -0.0931,  0.1733,  0.0309,
           0.1977,  0.3444, -0.0926, -0.2511,  0.1161,  0.0842, -0.0959,
           0.0996, -0.0221],
         [-0.6048,  0.3803,  0.5669,  0.3687,  0.1840,  0.4337,  0.3551,
           0.2703, -0.2130,  0.0049, -0.0772, -0.1377,  0.0443, -0.2824,
          -0.2769,  0.5257],
         [-0.0878,  0.4524,  0.3639,  0.3821,  0.2771, -0.1935,  0.1082,
          -0.0107, -0.4040, -0.0544, -0.0787, -0.1890,  0.3107, -0.3292,
           0.2937,  0.6249],
         [-0.0605,  0.1597,  0.2790,  0.1715,  0.0026,  0.0617,  0.0148,
           0.2289, -0.1894, -0.1701, -0.0315, -0.0761,  0.2557, -0.2790,
  



===== FORWARD PASS GPTBlock =====
Input tensor shape: torch.Size([1, 5, 16])
Attention output shape: torch.Size([1, 5, 16])
Attention output values:
 tensor([[[-0.9281, -0.6185, -0.6200,  0.0637,  0.2042,  0.7953,  0.4934,
          -0.8538, -0.2729,  0.3200,  0.3530, -0.0619,  0.5641, -0.3844,
           0.4562,  0.0455],
         [ 0.0887, -0.4910, -0.3405,  0.2276, -0.0073,  0.4907,  0.1419,
          -0.2056, -0.2369,  0.2991,  0.0270,  0.2408,  0.3458, -0.2494,
          -0.1236,  0.4249],
         [ 0.2936, -0.1846, -0.1150,  0.0176,  0.2602, -0.0943, -0.0831,
          -0.1513, -0.2719,  0.2654, -0.0218,  0.2148, -0.1572,  0.0498,
          -0.2274,  0.2091],
         [-0.2837, -0.0567, -0.2609, -0.1931,  0.3167, -0.1006, -0.0679,
          -0.3130, -0.2458,  0.1872,  0.2087,  0.0036, -0.3275,  0.2212,
           0.0785, -0.0915],
         [-0.2532, -0.0466, -0.2012, -0.1476,  0.3529, -0.2844, -0.0088,
          -0.3296, -0.1927,  0.2130,  0.1163, -0.0104, -0.4294,  0.1886,
  



===== FORWARD PASS GPTBlock =====
Input tensor shape: torch.Size([1, 6, 16])
Attention output shape: torch.Size([1, 6, 16])
Attention output values:
 tensor([[[-0.0173, -0.0520,  0.0578,  0.1463, -0.3406, -0.0839,  0.0198,
           0.4546,  0.0734, -0.5396, -0.3322,  0.3096,  0.6608, -0.3068,
           0.8802,  0.2424],
         [-0.4089, -0.2677, -0.0553, -0.0294, -0.0931,  0.1733,  0.0309,
           0.1977,  0.3444, -0.0926, -0.2511,  0.1161,  0.0842, -0.0959,
           0.0996, -0.0221],
         [-0.6048,  0.3803,  0.5669,  0.3687,  0.1840,  0.4337,  0.3551,
           0.2703, -0.2130,  0.0049, -0.0772, -0.1377,  0.0443, -0.2824,
          -0.2769,  0.5257],
         [-0.0878,  0.4524,  0.3639,  0.3821,  0.2771, -0.1935,  0.1082,
          -0.0107, -0.4040, -0.0544, -0.0787, -0.1890,  0.3107, -0.3292,
           0.2937,  0.6249],
         [-0.0605,  0.1597,  0.2790,  0.1715,  0.0026,  0.0617,  0.0148,
           0.2289, -0.1894, -0.1701, -0.0315, -0.0761,  0.2557, -0.2790,
  



===== FORWARD PASS GPTBlock =====
Input tensor shape: torch.Size([1, 6, 16])
Attention output shape: torch.Size([1, 6, 16])
Attention output values:
 tensor([[[-0.9281, -0.6185, -0.6200,  0.0637,  0.2042,  0.7953,  0.4934,
          -0.8538, -0.2729,  0.3200,  0.3530, -0.0619,  0.5641, -0.3844,
           0.4562,  0.0455],
         [ 0.0887, -0.4910, -0.3405,  0.2276, -0.0073,  0.4907,  0.1419,
          -0.2056, -0.2369,  0.2991,  0.0270,  0.2408,  0.3458, -0.2494,
          -0.1236,  0.4249],
         [ 0.2936, -0.1846, -0.1150,  0.0176,  0.2602, -0.0943, -0.0831,
          -0.1513, -0.2719,  0.2654, -0.0218,  0.2148, -0.1572,  0.0498,
          -0.2274,  0.2091],
         [-0.2837, -0.0567, -0.2609, -0.1931,  0.3167, -0.1006, -0.0679,
          -0.3130, -0.2458,  0.1872,  0.2087,  0.0036, -0.3275,  0.2212,
           0.0785, -0.0915],
         [-0.2532, -0.0466, -0.2012, -0.1476,  0.3529, -0.2844, -0.0088,
          -0.3296, -0.1927,  0.2130,  0.1163, -0.0104, -0.4294,  0.1886,
  

In [ ]:
# Buat vocab sederhana
vocab = ["hello", "world", "this", "is", "a", "test", "<PAD>", "<UNK>"]
tokenizer = SimpleTokenizer(vocab)

# Inisialisasi model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomGPT(
    vocab_size=len(vocab),
    embed_size=16,
    num_layers=2,
    heads=2,
    device=device,
    forward_expansion=4,
    dropout=0.1,
    max_length=10
).to(device)

# Contoh input
text = "hello world"
input_ids = tokenizer.encode(text).T.to(device)

# Generate teks baru
generated_ids = model.generate_text(input_ids, max_length=5)
generated_text = tokenizer.decode(generated_ids.squeeze(0).tolist())

print("Generated Text:", generated_text)



===== FORWARD PASS GPTBlock =====
Input tensor shape: torch.Size([1, 2, 16])
Attention output shape: torch.Size([1, 2, 16])
Attention output values:
 tensor([[[ 0.4976, -0.0425, -0.6015, -0.8495,  1.3402, -0.0253, -0.6126,
           0.2388,  0.5035, -0.1005,  0.5271,  0.4137,  0.6495,  0.3852,
           0.3827,  0.7076],
         [-0.1732,  0.2964,  0.4973, -0.1324,  0.7904,  0.1026, -0.2555,
           0.3387, -0.3192, -0.3305, -0.3001,  0.0357,  0.8668, -0.2059,
          -0.1829, -0.1117]]], grad_fn=<TransposeBackward0>)
After LayerNorm1 shape: torch.Size([1, 2, 16])
After LayerNorm1 values:
 tensor([[[ 0.2980, -0.1057, -2.5523, -0.7322, -0.2220, -0.7078, -0.1928,
           0.9957,  1.1922,  0.4540,  1.5406, -0.0144,  0.2068,  1.2288,
          -1.3077, -0.0813],
         [-0.7321,  1.3929,  0.1498,  0.8795,  0.6939, -0.4162, -1.0340,
          -0.4927, -1.8187, -0.1534, -1.4248,  2.0437, -0.2131, -0.2776,
           0.4598,  0.9431]]], grad_fn=<NativeLayerNormBackward0>)
Feed F

In [ ]:
import torch
import torch.nn as nn
from qiskit import QuantumCircuit
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit.primitives import StatevectorEstimator  # Ganti dari Estimator (Deprecated)
from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector

class QuantumNeuralNetwork(nn.Module):
    def __init__(self, num_features, num_qubits, reps=1):
        super(QuantumNeuralNetwork, self).__init__()

        self.num_features = num_features
        self.num_qubits = num_qubits  # Menggunakan jumlah qubit sesuai input

        # Gunakan StatevectorEstimator (V2) sebagai pengganti Estimator (V1)
        estimator = StatevectorEstimator()

        # Feature map & ansatz
        feature_map = ZZFeatureMap(self.num_qubits)
        ansatz = RealAmplitudes(self.num_qubits, reps=reps)

        # Quantum Circuit
        qc = QuantumCircuit(self.num_qubits)
        qc.compose(feature_map, inplace=True)
        qc.compose(ansatz, inplace=True)

        # Definisi QNN
        qnn = EstimatorQNN(
            circuit=qc,
            input_params=feature_map.parameters,
            weight_params=ansatz.parameters,
            input_gradients=True,
            estimator=estimator,
        )

        # Menghubungkan dengan PyTorch menggunakan TorchConnector
        self.qnn_layer = TorchConnector(qnn)

        # Fully connected layer setelah QNN
        self.fc = nn.Linear(1, 1)  # Output QNN adalah skalar, jadi Linear(1, 1)

    def forward(self, x):
        batch_size, seq_length, feature_dim = x.shape

        # Pastikan feature_dim sesuai dengan num_features
        if feature_dim != self.num_features:
            raise ValueError(f"Input feature_dim={feature_dim}, but expected {self.num_features}")

        # Ubah input agar sesuai untuk QNN
        x = x.view(batch_size * seq_length, feature_dim)

        x = self.qnn_layer(x)  # Proses di QNN (Output adalah skalar per input)

        x = self.fc(x)  # Masukkan ke Fully Connected Layer
        x = x.view(batch_size, seq_length, -1)  # Kembalikan ke ukuran semula

        return x


In [ ]:
# Contoh tensor input
x = torch.randn(1, 6, 16)  # (batch_size=1, seq_length=6, num_features=16)

# Membuat model dengan num_features=16 dan num_qubits=2 (bisa diubah)
model = QuantumNeuralNetwork(num_features=16, num_qubits=16)

# Forward pass
output = model(x)
print(output.shape)  # Seharusnya (1, 6, 1)

QiskitMachineLearningError: 'Invalid input dimension! Received torch.Size([6, 16]) and expected input compatible to 2'

In [ ]:
output

tensor([[[-0.5048],
         [-0.5093],
         [-0.5014],
         [-0.5109],
         [-0.5039],
         [-0.5010]]], grad_fn=<ViewBackward0>)